- # Visualizando os Leads que preencheram formulário:

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display
!pip install psycopg2-binary python-dotenv -q

load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")

def visualizar_leads_local():
    """
    Conecta ao banco de dados no Render e exibe os dados da tabela 'leads'
    em um formato de tabela (DataFrame).
    """
    if not DATABASE_URL:
        print("\nERRO: A variável DATABASE_URL não foi encontrada no seu arquivo .env.")
        print("Certifique-se de que o arquivo está na mesma pasta do seu notebook e que a variável está definida.")
        return

    conn = None
    try:
        import psycopg2 # Importa a biblioteca que acabamos de instalar
        
        print("\nConectando ao banco de dados PostgreSQL no Render...")
        conn = psycopg2.connect(DATABASE_URL)
        print("Conexão bem-sucedida!")

        # Comando SQL para selecionar todos os dados da tabela 'leads'
        sql_query = "SELECT * FROM leads ORDER BY data_ingestao DESC;"

        print("Buscando os leads cadastrados...")
        df = pd.read_sql_query(sql_query, conn)

        if df.empty:
            print("\nNenhum lead encontrado na tabela ainda.")
        else:
            print(f"\nExibindo {len(df)} leads encontrados:")
            display(df) # Exibe a tabela formatada no Jupyter

    except Exception as e:
        print(f"\nERRO: Não foi possível buscar os dados. Causa: {e}")

    finally:
        if conn:
            conn.close()
            print("\nConexão com o banco de dados foi fechada.")

if __name__ == '__main__':
    visualizar_leads_local()




[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Desktop\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



Conectando ao banco de dados PostgreSQL no Render...
Conexão bem-sucedida!
Buscando os leads cadastrados...


C:\Users\Desktop\AppData\Local\Temp\ipykernel_8724\447959727.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)



Exibindo 2 leads encontrados:


,id,nome,email,telefone,cargo,empresa,nascimento,data_admissao,data_ingestao,status_n8n
0,3,Leandro,leandro@teste.com.br,11963195328,Gerente,Lelis,None,None,2025-10-15 17:31:48.492947+00:00,PENDENTE
1,2,teste,teste@gmail.com,teste,teste,teste,None,None,2025-10-15 17:24:54.145508+00:00,PENDENTE



Conexão com o banco de dados foi fechada.


- # ENVIAR LEAD PARA N8N CLASSIFICAR!

In [6]:
import os
import json
import requests
from dotenv import load_dotenv
!pip install psycopg2-binary python-dotenv requests -q

print("\nCarregando variáveis de ambiente (DATABASE_URL e N8N_WEBHOOK_URL)...")
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")
N8N_WEBHOOK_URL = os.getenv("N8N_WEBHOOK_URL")

def enviar_leads_pendentes_para_n8n():
    """
    Busca leads com status 'PENDENTE' no banco de dados, envia para o n8n
    e atualiza o status para 'PROCESSADO' após o envio bem-sucedido.
    """
    # Validação das variáveis de ambiente
    if not DATABASE_URL or not N8N_WEBHOOK_URL:
        print("\nERRO: Verifique se as variáveis DATABASE_URL e N8N_WEBHOOK_URL estão definidas no seu arquivo .env.")
        return

    # Importa a biblioteca do DB aqui para garantir que foi instalada
    import psycopg2
    import psycopg2.extras

    conn = None
    try:
        print("\nConectando ao banco de dados PostgreSQL no Render...")
        conn = psycopg2.connect(DATABASE_URL)
        # Usamos DictCursor para obter os resultados como dicionários, o que facilita o envio
        cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        print("Conexão bem-sucedida!")

        # Busca apenas os leads que o n8n ainda não processou
        sql_query = "SELECT * FROM leads WHERE status_n8n = 'PENDENTE';"
        print("Buscando leads pendentes...")
        cur.execute(sql_query)
        leads_pendentes = cur.fetchall()

        if not leads_pendentes:
            print("\nNenhum lead novo para enviar ao n8n.")
            return

        print(f"\nEncontrados {len(leads_pendentes)} leads para enviar. Iniciando o processo...")
        leads_enviados_com_sucesso = 0

        for lead in leads_pendentes:
            lead_dict = dict(lead)
            lead_id = lead_dict['id']

            # Converte objetos de data/datetime para string no formato ISO, que é o padrão para JSON
            for key, value in lead_dict.items():
                if hasattr(value, 'isoformat'):
                    lead_dict[key] = value.isoformat()
            
            try:
                # Envia o lead para o webhook do n8n
                response = requests.post(N8N_WEBHOOK_URL, data=json.dumps(lead_dict), headers={'Content-Type': 'application/json'})
                
                if response.status_code == 200:
                    print(f"  - Lead ID {lead_id} enviado com sucesso para o n8n.")
                    
                    # Se o envio foi bem-sucedido, atualiza o status no banco de dados
                    update_sql = "UPDATE leads SET status_n8n = 'PROCESSADO' WHERE id = %s;"
                    cur.execute(update_sql, (lead_id,))
                    conn.commit() # Salva a alteração no banco
                    print(f"    -> Status do Lead ID {lead_id} atualizado para 'PROCESSADO'.")
                    leads_enviados_com_sucesso += 1
                else:
                    print(f"  - ERRO ao enviar lead ID {lead_id}. Status: {response.status_code}, Resposta: {response.text}")

            except requests.exceptions.RequestException as e:
                print(f"  - ERRO de conexão ao enviar lead ID {lead_id}: {e}")
        
        print(f"\nProcesso finalizado. {leads_enviados_com_sucesso} de {len(leads_pendentes)} leads foram enviados e atualizados.")

    except Exception as e:
        print(f"\nERRO GERAL: Ocorreu um problema durante a execução. Causa: {e}")
        if conn:
            conn.rollback() # Desfaz qualquer alteração pendente em caso de erro

    finally:
        if conn:
            cur.close()
            conn.close()
            print("\nConexão com o banco de dados foi fechada.")


# --- 4. EXECUTA A FUNÇÃO ---
if __name__ == '__main__':
    enviar_leads_pendentes_para_n8n()


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Desktop\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



Carregando variáveis de ambiente (DATABASE_URL e N8N_WEBHOOK_URL)...

Conectando ao banco de dados PostgreSQL no Render...
Conexão bem-sucedida!
Buscando leads pendentes...

Encontrados 1 leads para enviar. Iniciando o processo...
  - Lead ID 4 enviado com sucesso para o n8n.
    -> Status do Lead ID 4 atualizado para 'PROCESSADO'.

Processo finalizado. 1 de 1 leads foram enviados e atualizados.

Conexão com o banco de dados foi fechada.
